In [1]:
import tensorflow as tf

In [2]:
# Create tf dataset from a list
daily_sales_numbers = [21, 22, -108, 31, -1, 32, 34,31]

tf_dataset = tf.data.Dataset.from_tensor_slices(daily_sales_numbers)
tf_dataset

<TensorSliceDataset shapes: (), types: tf.int32>

In [3]:
for sales in tf_dataset:
    print(sales.numpy())

21
22
-108
31
-1
32
34
31


In [4]:
# also
for sales in tf_dataset.as_numpy_iterator():
    print(sales)

21
22
-108
31
-1
32
34
31


In [5]:
# Iterate through first n elements in tf dataset
for sales in tf_dataset.take(3):
    print(sales.numpy())

21
22
-108


In [6]:
# Filter sales numbers that are < 0
tf_dataset = tf_dataset.filter(lambda x: x>0)
for sales in tf_dataset.as_numpy_iterator():
    print(sales)

21
22
31
32
34
31


In [7]:
# Convert sales numbers from USA dollars ($) to Indian Rupees (INR) Assuming 1->75 conversation rate
tf_dataset = tf_dataset.map(lambda x: x*75)
for sales in tf_dataset.as_numpy_iterator():
    print(sales)

1575
1650
2325
2400
2550
2325


In [10]:
# batch of size 2
for sales in tf_dataset.batch(2):
    print(sales.numpy())

[1575 1650]
[2325 2400]
[2550 2325]


In [15]:
# all above operation in one
tf_dataset= tf.data.Dataset.from_tensor_slices(daily_sales_numbers)
tf_dataset= tf_dataset.filter(lambda x: x>0).map(lambda y: y*75).shuffle(2).batch(2)

for sales in tf_dataset.as_numpy_iterator():
    print(sales)

[1575 1650]
[2325 2400]
[2325 2550]


### Images

In [17]:
images_ds= tf.data.Dataset.list_files('datasets/*/*', shuffle=False)

for file in images_ds.take(3):
    print(file.numpy())

b'datasets\\cat\\00000001_000.jpg'
b'datasets\\cat\\00000001_005.jpg'
b'datasets\\cat\\00000001_008.jpg'


In [32]:
images_ds = images_ds.shuffle(200)
for file in images_ds.take(3):
    print(file.numpy())

b'datasets\\dog\\dog.4019.jpg'
b'datasets\\flower_photos\\daisy'
b'datasets\\flower_photos\\tulips'


In [26]:
class_name=['cat', 'dog']

In [27]:
image_count= len(images_ds)
image_count

30

In [29]:
train_size= int(image_count*0.8)

train_ds= images_ds.take(train_size)
# skip 1st 80% of data
test_ds= images_ds.skip(train_size)

In [31]:
print(len(train_ds))
print(len(test_ds))

24
6


In [35]:
s= 'datasets\\dog\\dog.4019.jpg'
s.split('\\')[-2]

'dog'

In [48]:
# getting label from file path
def get_label(file_path):
    import os
    parts = tf.strings.split(file_path, os.path.sep)
    return parts[-2]

In [49]:
def process_image(file_path):
    label = get_label(file_path)
    img = tf.io.read_file(file_path) # load the raw data from the file as a string
    img = tf.image.decode_jpeg(img)
    img = tf.image.resize(img, [128, 128])
    return img, label

In [54]:
img, label = process_image("datasets\\dog\\dog.4019.jpg")
img.numpy()[:2]

array([[[ 58.293213,  56.447388,  74.97864 ],
        [ 65.70776 ,  52.732788,  73.10315 ],
        [ 73.403564,  47.86194 ,  68.36182 ],
        [ 72.305786,  36.79016 ,  57.026733],
        [ 62.270386,  32.234375,  40.234375],
        [ 59.016235,  32.96216 ,  41.96216 ],
        [ 54.004272,  32.785522,  41.606567],
        [ 52.765747,  35.687622,  43.609497],
        [ 42.21045 ,  31.55017 ,  54.36267 ],
        [ 70.38916 ,  68.748535,  93.15442 ],
        [ 82.4375  ,  94.015625, 121.984375],
        [ 83.60071 , 107.47742 , 138.03906 ],
        [ 86.      , 115.      , 145.      ],
        [ 91.234375, 118.46875 , 144.82544 ],
        [ 90.71326 , 117.85388 , 134.01013 ],
        [ 75.140625, 102.140625,  96.953125],
        [ 83.42786 , 109.52161 , 100.38098 ],
        [ 95.330444, 122.330444, 113.611694],
        [100.43884 , 131.9701  , 119.20447 ],
        [101.31531 , 134.65906 , 120.90906 ],
        [105.111084, 135.97131 , 122.67273 ],
        [108.39636 , 136.92188 , 1

In [55]:
train_ds = train_ds.map(process_image)
test_ds = test_ds.map(process_image)

In [56]:
for image, label in train_ds.take(1):
    print("****",image)
    print("****",label)

**** tf.Tensor(
[[[106.78876   89.68036   64.39789 ]
  [102.55493   68.12402   56.946472]
  [108.31091   74.05487   56.374023]
  ...
  [137.357    137.99695  137.21893 ]
  [136.55676  138.49908  135.38373 ]
  [136.07452  137.51501  133.47565 ]]

 [[108.271545  92.70422   70.61853 ]
  [103.26013   79.97406   69.6004  ]
  [119.632935  71.57733   68.49042 ]
  ...
  [135.24677  137.13416  133.19019 ]
  [134.75665  134.89178  132.82422 ]
  [135.33685  137.33685  136.31158 ]]

 [[106.01166   91.228516  78.93378 ]
  [109.42743   86.06024   77.0036  ]
  [107.87616   61.423462  62.165955]
  ...
  [136.73346  138.51099  135.60828 ]
  [134.89453  136.36328  133.474   ]
  [135.48676  137.16016  132.33594 ]]

 ...

 [[ 82.94592   49.252136  44.691833]
  [ 48.432007  27.192627  20.954224]
  [ 91.17236   71.0318    69.9328  ]
  ...
  [ 14.183228  12.758545   9.326416]
  [  9.205261  11.740967  10.311279]
  [ 29.100037  16.779724  14.390625]]

 [[ 85.57745   60.132996  27.716187]
  [101.84711   68.734

In [57]:
def scale(image, label):
    return image/255, label

In [58]:
train_ds = train_ds.map(scale)

In [60]:
for image, label in train_ds:
    print("-->Image: ",image.numpy()[0][0])
    print("-->Label: ",label.numpy())

-->Image:  [0.41877943 0.35168767 0.25254074]
-->Label:  b'cat'
-->Image:  [0.5773308  0.5848063  0.54810286]
-->Label:  b'cat'
-->Image:  [0.3852539  0.35289282 0.31497756]
-->Label:  b'cat'
-->Image:  [0.6978236  0.6022413  0.66325873]
-->Label:  b'cat'
-->Image:  [0.22567402 0.31038603 0.40356755]
-->Label:  b'cat'
-->Image:  [0.24604014 0.27051166 0.24078585]
-->Label:  b'cat'
-->Image:  [0.5852482  0.48452818 0.4882353 ]
-->Label:  b'cat'
-->Image:  [0.03151042 0.03935355 0.03543198]
-->Label:  b'cat'
-->Image:  [0.43677667 0.4328551  0.41716883]
-->Label:  b'cat'
-->Image:  [0.6924003  0.69632185 0.6649493 ]
-->Label:  b'cat'
-->Image:  [0.94879746 0.9077474  0.921358  ]
-->Label:  b'cat'
-->Image:  [0.19803922 0.20686275 0.22352941]
-->Label:  b'cat'
-->Image:  [0.03267344 0.03659501 0.04443814]
-->Label:  b'cat'
-->Image:  [0.35453317 0.314747   0.24282023]
-->Label:  b'dog'
-->Image:  [0.32020253 0.27706525 0.2970714 ]
-->Label:  b'dog'
-->Image:  [0.35407045 0.07956064 0.0403